In [ ]:
# ----- DATA EXPLORATION AND CLEANING -----
# Purpose: explore the raw data and prepare it for cleaning

In [ ]:
# --- STAGE 1: LOAD AND INITIAL DATA STRUCTURE CHECKS ---

# Import libraries
import pandas as pd
import numpy as np  
import seaborn as sns
import matplotlib.pyplot as plt 

# 1. Load Data
df = pd.read_csv("../data/raw/global_findex.csv")

# 2. View Raw Data Sample
print("--- Data Head and Tail (Initial Sample) ---")
print(df.head())
print(df.tail())

# 3. Check Unique Values in Key Columns
print("\nUnique Countries/aggregates in 'countrynewwb':")
print(df['countrynewwb'].unique())

print("\nUnique income groups in 'incomegroupwb24':")
print(df['incomegroupwb24'].unique())

print("\nUnique regions in 'regionwb24_hi':")
print(df['regionwb24_hi'].unique())


# 3. Check Data Dimensions.
print("\n--- 3. Data Shape ---")
# Check number of rows (observations) and columns (variables)
print(f"Dataset has: {df.shape[0]} rows and {df.shape[1]} columns") 

# 4. Data Types and Non-Null Counts (Primary Missing Value Check)
print("\n--- 4. Data Types and Missing Values Summary (df.info()) ---")
df.info()

# 5. Descriptive Statistics (General overview of numeric columns)
print(f"\n--- 5. Descriptive Statistics (Numeric only) ---")
print(df.describe())

# 6. Calculation of Missing Values (Percentage)
print("\n--- 6. Detailed Missing Value Count and Percentage ---")
missing_data = df.isnull().sum()
# Calculate percentage
missing_percentage = (missing_data[missing_data > 0] / len(df)) * 100

# Create a summary table for missing data that is not 0
missing_summary = pd.DataFrame({
    'Missing_Count': (missing_data[missing_data > 0]),
    'Missing_Percent': missing_percentage.round(2)
})

print(missing_summary)
# Look for columns with a high percentage (>= 50%) as they are candidate for removal.

# 7. Check for Duplicates (Integrity Check)
print("\n--- 7. Duplicate Rows Check ---")
# Counts the number of full row duplicates
num_duplicates = df.duplicated().sum()
print(f"Number of fully duplicated rows: {num_duplicates}")
# Cleaning Action (if duplicates > 0): df.drop_duplicates(inplace=True)


C:\Users\Aidid\AppData\Local\Temp\ipykernel_7424\1184767123.py:10: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/global_findex.csv")


--- Data Head and Tail (Initial Sample) ---
  countrynewwb codewb  year   pop_adult  \
0  Afghanistan    AFG  2011  14575546.0   
1      Albania    ALB  2011   2281010.0   
2      Algeria    DZA  2011  26251587.0   
3       Angola    AGO  2011  12779501.0   
4    Argentina    ARG  2011  30685516.0   

                                       regionwb24_hi      incomegroupwb24  \
0                 South Asia (excluding high income)           Low income   
1      Europe & Central Asia (excluding high income)  Upper middle income   
2  Middle East & North Africa (excluding high inc...  Lower middle income   
3         Sub-Saharan Africa (excluding high income)  Lower middle income   
4  Latin America & Caribbean (excluding high income)  Upper middle income   

  group group2  account_t_d  fiaccount_t_d  ...  con12m_s  con26lm_s  \
0   all    all     0.090050       0.090050  ...       NaN        NaN   
1   all    all     0.282681       0.282681  ...       NaN        NaN   
2   all    all    

In [25]:
# --- PRIORITY CLEANING: FILTERING AGGREGATES AND DROPPING NEAR-EMPTY COLUMNS ---
print("="*60)
print("FILTERING AGGREGATES AND DROPPING NEAR-EMPTY COLUMNS")
print("="*60)

# 1. IDENTIFY AND FILTER AGGREGATE ROWS
# The Findex dataset includes summary rows (e.g., 'world', 'Low income')
# Alongside individual country data. These must be removed to prevent bias and double-counting
# in country-level analysis.

# Define the list of aggregate entities to be excluded from country-level analysis.
Aggregate_terms = ['world', 'Developing economies', 'East Asia & Pacific (excluding high income)',
                   'Europe & Central Asia (excluding high income)', 'Middle East & North Africa (excluding high income)',
                   'Sub-Saharan Africa (excluding high income)', 'Latin America & Caribbean (excluding high income)',
                   'South Asia', 'High income', 'Low income', 'Lower middle income', 'Upper middle income']

# Create a boolean mask: True for rows where 'countrynewwb' is NOT in the Aggregate_terms list.
country_mask = ~df['countrynewwb'].isin(Aggregate_terms)

# Apply the filter to create a new DataFrame containing only individual country data
df_filtered = df[country_mask].copy()

print("--- Post-Filtering Shape Check ---")
print(f"\nOriginal rows: {df.shape[0]}, Rows after removing aggregates: {df_filtered.shape[0]}")
print(f"Number of aggregate rows removed: {df.shape[0] - df_filtered.shape[0]}")

# 2. REMOVE COLUMNS WITH EXCESSIVE MISSING DATA
print("\n--- CHECKING AND REMOVING EXCESSIVE MISSING DATA ---")
# Prevent truncation
pd.set_option('display.max_columns', None) # show all columns in summary missing values table
pd.set_option('display.max_rows', None) # show all rows in summary missing value table
pd.set_option('display.width', 1000) # Allow wide output to prevent wrapping

# Recalculate the missing values and percentage based on the new, smaller (filtered) dataset.
missing_values_filtered = df_filtered.isnull().sum()
missing_percentage_filtered = (df_filtered.isnull().sum() / len(df_filtered)) * 100

# Creating a summary table for missing 
missing_summary_table = pd.DataFrame({
    'Missing Count': missing_values_filtered,
    'Missing Percentage': missing_percentage_filtered.round(2)
})

# Filter to show ONLY columns with missing data and sort by percentage (descending).
# This is the table you need to review to decide on your threshold.
missing_summary_sorted = missing_summary_table[missing_summary_table['Missing Count'] > 0].sort_values(by='Missing Percentage', ascending=False)

print(missing_summary_sorted)

# Set a threshold for removal(columns with 90.0% or more missing values are useless)
missing_threshold = 90.0

# Identify columns where the missing percentage meets or exceeds the threshold.
cols_to_drop = missing_percentage_filtered[missing_percentage_filtered >= missing_threshold].index.tolist()

# Drop the identified columns from the filtered DataFrame
df_cleaned = df_filtered.drop(columns=cols_to_drop)

print("\n--- Post-Column Drop Shape Check ---")
print(f"Columns dropped due to >={missing_threshold}% missing data: {len(cols_to_drop)}")
print(f"New DataFrame shape (Rows, Columns): {df_cleaned.shape}")

# Overwrite the main DataFrame variable (df) to use the cleaned version moving forward.
df = df_cleaned






FILTERING AGGREGATES AND DROPPING NEAR-EMPTY COLUMNS
--- Post-Filtering Shape Check ---

Original rows: 7880, Rows after removing aggregates: 7880
Number of aggregate rows removed: 0

--- CHECKING AND REMOVING EXCESSIVE MISSING DATA ---
                   Missing Count  Missing Percentage
fin24bor                    7078               89.82
fh2a                        7073               89.76
fin6m                       7049               89.45
fin45e                      7029               89.20
fin9b                       7013               89.00
con26d                      7004               88.88
con30e                      6992               88.73
fin5m                       6989               88.69
fin22h                      6983               88.62
fin9a                       6977               88.54
fin17b                      6976               88.53
fin22c                      6976               88.53
con30a                      6956               88.27
con30d               

In [24]:
# --- STAGE 2: CONSISTENCY AND CATEGORICAL VALUE CHECKS ---
print("="*60)
print("STAGE 2: CONSISTENCY AND CATEGORICAL VALUE CHECKS")
print("="*60)

# 1. Check the structure of the core grouping variables
print("\n--- 1. Unique Values and Counts in Grouping Variables ---")
print("\nValue Counts for 'group':")
print(df['group'].value_counts())

print("\nValue Counts for 'group2':")
print(df['group2'].value_counts())

# 2. Check the distribution of time/year
print("\n--- 2. Unique Years in Dataset ---")
print(df['year'].value_counts(dropna=False).sort_index())

# 3. Check the structure of key demographics column
print("\nUnique values in 'incomegroupwb24':")
print(df['incomegroupwb24'].value_counts(dropna=False))

# Check the region column 
print("\nUnique values in 'regionwb24':")
print(df['regionwb24_hi'].value_counts(dropna=False))

# 4. Re-check Missing Data Summary
# This step helps identify the columns still above 50% missing for future special handling.
print("\n--- 4. Missing Data Summary for Remaining Columns (to be handled later) ---")

# Calculate missing percentages for the remaining columns
remaining_missing_data = df.isnull().sum()
remaining_missing_percentage = (remaining_missing_data / len(df)) * 100

# Create and print the summary table for columns with > 0% missingness
summary_table_final = pd.DataFrame({
    'Missing count': remaining_missing_data,
    'Missing percent': remaining_missing_percentage.round(2)
})

# Filter only columns that still have missing data and sort by percentage
# Limiting the display to the top 20 problematic columns for focus.
missing_summary_final_sorted = summary_table_final[summary_table_final['Missing count'] > 0].sort_values(by='Missing percent', ascending=False).head(20)

print("Top 20 Columns with Remaining Missing Data:")
print(missing_summary_final_sorted)

print("\n"+"="*60)
print("Decision Point Reminder:")
print("The remaining missing data (up to ~80% in the top rows) will be handled during the final Feature Engineering stage.")
print("="*60)





STAGE 2: CONSISTENCY AND CATEGORICAL VALUE CHECKS

--- 1. Unique Values and Counts in Grouping Variables ---

Value Counts for 'group':
group
gender        1388
age_cat       1388
education     1386
laborforce    1368
income        1362
all            711
urbanicity     277
Name: count, dtype: int64

Value Counts for 'group2':
group2
all                      711
men                      694
women                    694
ages 15-24               694
age 25+                  694
secondary edu or more    693
prim edu or less         693
in laborforce            684
out of laborforce        684
poorest 40%              681
richest 60%              681
urban                    139
rural                    138
Name: count, dtype: int64

--- 2. Unique Years in Dataset ---
year
2011    1384
2014    1554
2017    1602
2021    1351
2022     176
2024    1813
Name: count, dtype: int64

Unique values in 'incomegroupwb24':
incomegroupwb24
High income            2630
Lower middle income    2211
Upper m